Nguyễn Hoài Phú - 31221024738

Số máy 02

29092004

# CHO BÀN CỜ KÍCH THƯỚC 5×5. HÃY SỬ DỤNG THƯ VIỆN SIMPLEAI ĐỂ TÌM RA MỘT PHƯƠNG ÁN XẾP 5 QUÂN HẬU VÀO BÀN CỜ NÓI TRÊN ĐỂ KHÔNG CÓ CẶP HẬU NÀO ĂN NHAU[*] , BẰNG:

[*] Một cặp quân hậu sẽ ăn nhau nếu nằm trên đường thẳng (ngang hoặc dọc) hoặc
đường chéo (chính hoặc phụ).


### Câu 1: XÂY DỰNG BÀI TOÁN TÌM KIẾM VÀ SỬ DỤNG THUẬT TOÁN GENETIC ALGORITHM ĐỂ GIẢI.

In [ ]:
from simpleai.search import SearchProblem, genetic
import random

class NQueensProblem(SearchProblem): # Kế thừa SearchProblem
    def __init__(self, N=5):
        super(NQueensProblem, self).__init__()

        # Bàn cờ 5x5
        self.N = N

        # Khởi tạo vị trí ngẫu nhiên cho các quân cờ
        self.initial_state = tuple([random.randint(0, N-1) for _ in range(N)])

    # Tạo trạng thái ngẫu nhiên cho quá trình tạo thế hệ ban đầu của GA Algorithm
    def generate_random_state(self):
        """Một danh sách N số nguyên, mỗi số từ 0 đến N-1."""
        return tuple([random.randint(0, self.N-1) for _ in range(self.N)])

    # Hàm lai ghép: Phần đầu của state1, đuôi của state2
    def crossover(self, state1, state2):
        """Chọn một điểm cắt ngẫu nhiên và kết hợp hai trạng thái."""
        cut_point = random.randint(1, self.N-1)
        child = list(state1[:cut_point]) + list(state2[cut_point:])
        return tuple(child)

    # Đột biến để sinh ra cá thể mới
    def mutate(self, state):
        """Thay đổi ngẫu nhiên một vị trí trong trạng thái."""
        state = list(state)
        pos = random.randint(0, self.N-1)
        state[pos] = random.randint(0, self.N-1)
        return tuple(state)

    # Hàm đánh giá độ phù hợp (fitness)
    def value(self, state):
        """Số cặp hậu không ăn nhau."""
        non_attacking_pairs = 0
        for i in range(self.N):
            for j in range(i + 1, self.N):
                # Kiểm tra cùng hàng
                if state[i] != state[j]:
                    # Kiểm tra cùng đường chéo
                    if abs(state[i] - state[j]) != abs(i - j):
                        non_attacking_pairs += 1
        return non_attacking_pairs

def print_board(state, N=5):
    """In bàn cờ với các quân hậu."""
    board = [['.' for _ in range(N)] for _ in range(N)]
    for col, row in enumerate(state):
        board[row][col] = 'Q'
    for row in board:
        print(' '.join(row))

def main():
    problem = NQueensProblem(N=5)

    # Chạy giải thuật di truyền
    result = genetic(problem, population_size=100, mutation_chance=0.1, iterations_limit=1000)

    print("Trạng thái cuối cùng:", result.state)
    print("Độ phù hợp:", result.value)
    print("Bàn cờ:")
    print_board(result.state)

    # Kiểm tra lời giải
    max_pairs = (5 * (5 - 1)) // 2  # Số cặp hậu tối đa: C(5,2) = 10
    if result.value == max_pairs:
        print("Lời giải hợp lệ: Không có cặp hậu nào ăn nhau.")
    else:
        # Có trường hợp nào đó 2 cặp hậu ăn nhau
        print("Lời giải không hợp lệ: Có cặp hậu ăn nhau.")

if __name__ == "__main__":
    main()

Trạng thái cuối cùng: (3, 0, 2, 4, 1)
Độ phù hợp: 10
Bàn cờ:
. Q . . .
. . . . Q
. . Q . .
Q . . . .
. . . Q .
Lời giải hợp lệ: Không có cặp hậu nào ăn nhau.


### Câu 2: HÌNH THÀNH BÀI TOÁN DƯỚI DẠNG BÀI TOÁN CÓ RÀNG BUỘC CSP VÀ SỬ DỤNG THUẬT TOÁN BACKTRACK ĐỂ GIẢI.

In [ ]:
from simpleai.search import CspProblem, backtrack
import itertools

def constraint_different_rows_and_no_diagonal(vars_tuple, values_tuple):
    """Ràng buộc: hai quân hậu không cùng hàng và không trên cùng đường chéo."""
    col1, col2 = vars_tuple  # Các cột (C0, C1, ...)
    row1, row2 = values_tuple  # Các hàng tương ứng
    col_diff = abs(col1 - col2)
    row_diff = abs(row1 - row2)
    # Không cùng hàng (row1 != row2) và không trên đường chéo (row_diff != col_diff)
    return row1 != row2 and row_diff != col_diff

def print_board(solution, N=5):
    """In bàn cờ với các quân hậu."""
    board = [['.' for _ in range(N)] for _ in range(N)]
    for col, row in solution.items():
        board[row][col] = 'Q'
    for row in board:
        print(' '.join(row))

def main():
    # Định nghĩa bài toán CSP
    N = 5

    # Các biến: C0, C1, C2, C3, C4 (các cột)
    variables = [0, 1, 2, 3, 4]

    # Miền giá trị cho mỗi biến: các hàng 0, 1, 2, 3, 4
    domains = {var: list(range(N)) for var in variables}

    # Ràng buộc: tất cả các cặp biến phải thỏa mãn ràng buộc về hàng và đường chéo
    constraints = []
    for var1, var2 in itertools.combinations(variables, 2):
        constraints.append(((var1, var2), constraint_different_rows_and_no_diagonal))

    # Tạo bài toán CSP
    problem = CspProblem(variables, domains, constraints)

    # Giải bài toán bằng Backtracking
    solution = backtrack(problem)

    if solution:
        print("Lời giải:", solution)
        print("Bàn cờ:")
        print_board(solution)
    else:
        print("Không tìm được lời giải.")

if __name__ == "__main__":
    main()

Lời giải: {0: 0, 1: 2, 2: 4, 3: 1, 4: 3}
Bàn cờ:
Q . . . .
. . . Q .
. Q . . .
. . . . Q
. . Q . .
